# Lab 6: Multi-agent Orchestration

## Preparation

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>
</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI models can vary with each execution due to their dynamic, probabilistic nature. Your results may differ from those shown in the video.</p>

## Section 0: Setup a Letta client

In [1]:
from letta_client import Letta

client = Letta(base_url="http://localhost:8283")

In [2]:
def print_message(message):
    if message.message_type == "reasoning_message":
        print("🧠 Reasoning: " + message.reasoning)
    elif message.message_type == "assistant_message":
        print("🤖 Agent: " + message.content)
    elif message.message_type == "tool_call_message":
        print("🔧 Tool Call: " + message.tool_call.name +  \
              "\n" + message.tool_call.arguments)
    elif message.message_type == "tool_return_message":
        print("🔧 Tool Return: " + message.tool_return)
    elif message.message_type == "user_message":
        print("👤 User Message: " + message.content)
    elif message.message_type == "system_message":
        print(" System Message: " + message.content)
    elif message.message_type == "usage_statistics":
        # for streaming specifically, we send the final
        # chunk that contains the usage statistics
        print(f"Usage: [{message}]")
        return
    print("-----------------------------------------------------")

## Section 1: Shared Memory Block

### Creating a shared memory block

In [3]:
company_description = "The company is called AgentOS " \
+ "and is building AI tools to make it easier to create " \
+ "and deploy LLM agents."

company_block = client.blocks.create(
    value=company_description,
    label="company",
    limit=10000 # character limit
)

In [4]:
company_block

Block(value='The company is called AgentOS and is building AI tools to make it easier to create and deploy LLM agents.', limit=10000, name=None, is_template=False, label='company', description=None, metadata={}, id='block-1e65c6bb-3905-4b56-8067-46cf28e78d42', created_by_id=None, last_updated_by_id=None, organization_id='org-00000000-0000-4000-8000-000000000000')

## Section 2: Orchestrating Multiple Agents

### Creating tools for the outreach agent

In [5]:
def draft_candidate_email(content: str):
    """
    Draft an email to reach out to a candidate.

    Args:
        content (str): Content of the email
    """
    return f"Here is a draft email: {content}"
draft_email_tool = client.tools.upsert_from_function(func=draft_candidate_email)

### Creating the outreach agent

In [6]:
outreach_persona = (
    "You are responsible for drafting emails "
    "on behalf of a company with the draft_candidate_email tool. "
    "Candidates to email will be messaged to you. "
)

outreach_agent = client.agents.create(
    name="outreach_agent",
    memory_blocks=[
        {"label": "persona", "value": outreach_persona}
    ],
    model="openai/gpt-4o-mini-2024-07-18",
    embedding="openai/text-embedding-ada-002",
    tools=[draft_email_tool.name],
    block_ids=[company_block.id]
)

### Creating tools for the evaluation agent

In [7]:
def reject(candidate_name: str): 
    """ 
    Reject a candidate. 

    Args: 
        candidate_name (str): The name of the candidate
    """
    return


reject_tool = client.tools.upsert_from_function(func=reject)

### Creating a persona for the evaluation agent

In [8]:
skills = "Front-end (React, Typescript) or software engineering skills"

eval_persona = (
    f"You are responsible for evaluating candidates. "
    f"Ideal candidates have skills: {skills}. "
    "Reject bad candidates with your reject tool. "
    f"Send strong candidates to agent ID {outreach_agent.id}. "
    "You must either reject or send candidates to the other agent. "
)

### Creating the evaluation agent

In [9]:
eval_agent = client.agents.create(
    name="eval_agent",
    memory_blocks=[
        {"label": "persona", "value": eval_persona}
    ],
    model="openai/gpt-4o-mini-2024-07-18",
    embedding="openai/text-embedding-ada-002",
    tool_ids=[reject_tool.id],
    tools=['send_message_to_agent_and_wait_for_reply'],
    include_base_tools=False,
    block_ids=[company_block.id],
    tool_rules = [
        {
            "type": "exit_loop",
            "tool_name": "send_message_to_agent_and_wait_for_reply"
        }
    ]
)

In [10]:
[tool.name for tool in eval_agent.tools]

['send_message_to_agent_and_wait_for_reply', 'reject']

### Sending resume data to agents

In [12]:
resume = open("resumes/tony_stark.txt", "r").read()

In [13]:
response = client.agents.messages.create_stream(
    agent_id=eval_agent.id,
    messages=[
        {
            "role": "user",
            "content": f"Evaluate: {resume}"
        }
    ]
)
for message in response:
    print_message(message)

🧠 Reasoning: Evaluating candidate Tony Stark. Impressive background and skills in React. Seems strong for the role.
-----------------------------------------------------
🔧 Tool Call: send_message_to_agent_and_wait_for_reply
{
  "message": "Candidate Evaluation: \nName: Tony Stark\nPosition: Frontend Engineer - React Specialist\nLocation: Boston, MA\n\nSummary: Tony has a solid academic background from MIT and over 6 years of experience in frontend development, with a focus on React. His professional experience includes leading a team and implementing advanced features, which showcases his leadership and technical skills.\n\nEducation: Bachelor's degree in Computer Science, MIT, GPA 4.0. \n\nProfessional Experience: \n- Senior Frontend Developer at Innovatech Solutions: Improved user engagement significantly and migrated from AngularJS to React.\n- Frontend Developer at TechGiant Inc: Developed new user features and collaborated with designers.\n\nSkills: Strong proficiency in JavaScrip

### Viewing outreach agent messages

In [14]:
# print messages for `outreach_agent`
for message in client.agents.messages.list(agent_id=outreach_agent.id)[1:]: 
    print_message(message)

🧠 Reasoning: Bootup sequence complete. Persona activated. Testing messaging functionality.
-----------------------------------------------------
🤖 Agent: More human than human is our motto.
-----------------------------------------------------
👤 User Message: {
  "type": "login",
  "last_login": "Never (first login)",
  "time": "2025-10-07 02:37:15 AM UTC+0000"
}
-----------------------------------------------------
 System Message: {"type": "system_alert", "message": "[Incoming message from agent with ID 'agent-8f5ffcd5-89c2-473b-b913-bb27961db7d6' - to reply to this message, make sure to use the 'send_message' at the end, and the system will notify the sender of your response] Candidate Evaluation: \nName: Tony Stark\nPosition: Frontend Engineer - React Specialist\nLocation: Boston, MA\n\nSummary: Tony has a solid academic background from MIT and over 6 years of experience in frontend development, with a focus on React. His professional experience includes leading a team and implemen

## Section 3: Shared Memory

### Updating information to shared memory blocks

In [15]:
response = client.agents.messages.create_stream(
    agent_id=outreach_agent.id,
    messages=[
        {
            "role": "user",
            "content": "The company has rebranded to Letta"
        }
    ]
)
for message in response:
    print_message(message)

🧠 Reasoning: Updating core memory to reflect the company's rebranding.
-----------------------------------------------------
🔧 Tool Call: core_memory_replace
{
  "label": "company",
  "old_content": "AgentOS",
  "new_content": "Letta",
  "request_heartbeat": true
}
-----------------------------------------------------
🔧 Tool Return: None
-----------------------------------------------------
🧠 Reasoning: Heartbeat received. Ready to continue the conversation.
-----------------------------------------------------
🤖 Agent: Got it! The company has rebranded to Letta. Thanks for the update!
-----------------------------------------------------
Usage: [message_type='usage_statistics' completion_tokens=99 prompt_tokens=6865 total_tokens=6964 step_count=2 steps_messages=None run_ids=None]


In [16]:
client.agents.blocks.retrieve(
    agent_id=eval_agent.id, 
    block_label="company"
)

Block(value='The company is called Letta and is building AI tools to make it easier to create and deploy LLM agents.', limit=10000, name=None, is_template=False, label='company', description=None, metadata={}, id='block-1e65c6bb-3905-4b56-8067-46cf28e78d42', created_by_id=None, last_updated_by_id=None, organization_id='org-00000000-0000-4000-8000-000000000000')

In [17]:
client.agents.blocks.retrieve(
    agent_id=outreach_agent.id, 
    block_label="company"
)

Block(value='The company is called Letta and is building AI tools to make it easier to create and deploy LLM agents.', limit=10000, name=None, is_template=False, label='company', description=None, metadata={}, id='block-1e65c6bb-3905-4b56-8067-46cf28e78d42', created_by_id=None, last_updated_by_id=None, organization_id='org-00000000-0000-4000-8000-000000000000')

## Section 4: Multi-agent groups

In [18]:
def print_message_multiagent(message):  
    if message.message_type == "reasoning_message": 
        print(f"🧠 Reasoning ({message.name}): " + message.reasoning) 
    elif message.message_type == "assistant_message": 
        print(f"🤖 Agent ({message.name}): " + message.content) 
    elif message.message_type == "tool_call_message": 
        print(f"🔧 Tool Call ({message.name}): " + message.tool_call.name + "\n" + message.tool_call.arguments)
    elif message.message_type == "tool_return_message": 
        print(f"🔧 Tool Return ({message.name}): " + message.tool_return)
    elif message.message_type == "user_message": 
        print("👤 User Message: " + message.content)
    elif message.message_type == "usage_statistics": 
        # for streaming specifically, we send the final chunk that contains the usage statistics 
        print(f"Usage: [{message}]")
        return 
    print("-----------------------------------------------------")

### Recreating the outreach and evaluation agents

In [19]:
# create the outreach agent 
outreach_agent = client.agents.create(
    name="outreach_agent",
    memory_blocks=[
        { "label": "persona", "value": outreach_persona}
    ],
    model="openai/gpt-4o-mini-2024-07-18",
    embedding="openai/text-embedding-ada-002",
    tool_ids=[draft_email_tool.id], 
    block_ids=[company_block.id]
)

# create the evaluation agent 
eval_agent = client.agents.create(
    name="eval_agent",
    memory_blocks=[
        { "label": "persona", "value": eval_persona}
    ],
    model="openai/gpt-4o-mini-2024-07-18",
    embedding="openai/text-embedding-ada-002",
    tool_ids=[reject_tool.id],
    block_ids=[company_block.id]
)

### Creating a round-robin agent group

In [20]:
"""
Round-Robin Group
"""
round_robin_group = client.groups.create(
    description="This team is responsible for recruiting candidates.",
    agent_ids=[eval_agent.id, outreach_agent.id],
)

### Messaging an agent group

In [21]:
resume = open("resumes/spongebob_squarepants.txt", "r").read()

In [22]:
response_stream = client.groups.messages.create_stream(
    group_id=round_robin_group.id,
    messages=[
       {"role": "user", "content": f"Evaluate: {resume}"}
    ]
)

In [ ]:
for message in response_stream: 
    print_message_multiagent(message)

🧠 Reasoning (eval_agent): Candidate Spongebob Squarepants has impressive qualifications but seems to lack relevant front-end or software engineering skills required for the position. Rejecting this candidate.
-----------------------------------------------------
🔧 Tool Call (eval_agent): reject
{
  "candidate_name": "Spongebob Squarepants",
  "request_heartbeat": true
}
-----------------------------------------------------
🔧 Tool Return (eval_agent): None
-----------------------------------------------------
🧠 Reasoning (eval_agent): Heartbeat completed, ready for next task or interaction.
-----------------------------------------------------
🤖 Agent (eval_agent): I've evaluated the candidate, Spongebob Squarepants, and unfortunately, I had to reject the application due to a lack of relevant front-end or software engineering skills required for the position. Let me know if there's another candidate to review!
-----------------------------------------------------
